<table>
    <tr>
      <td>PYTHON PARA HUMANIDADES DIGITALES - Introducción al análisis y procesamiento de textos con Python</td>
      <td>Ediciones RC</td>
    </tr>
    <tr><td>Rafael Caballero Roldán, Enrique Martín Martín y Adrián Riesco Rodríguez</td>
    </tr>
</table>

# Capítulo 5: Análisis semántico

## Índice
[Bibliotecas importadas](#bib_imp)<br>
[Introducción](#intro)<br>
[Análisis de sentimiento](#sentimiento)<br>
&nbsp;&nbsp;&nbsp;&nbsp;  [Definición de funciones](#sent_fun)<br>
&nbsp;&nbsp;&nbsp;&nbsp;  [Ejecución](#sent_ejec)<br>
[Semejanza semántica](#semejanza)<br>
&nbsp;&nbsp;&nbsp;&nbsp;  [Definición de funciones](#semejanza_fun)<br>
&nbsp;&nbsp;&nbsp;&nbsp;  [Ejecución](#semejanza_ejec)<br>
[Modelado de temas](#model)<br>
&nbsp;&nbsp;&nbsp;&nbsp;  [Definición de funciones](#model_fun)<br>
&nbsp;&nbsp;&nbsp;&nbsp;  [Ejecución](#model_ejec)<br>

<a name="bib_imp"></a>
### Bibliotecas importadas

In [ ]:
!pip install spacy==3.4.2
!pip install pysentimiento==0.5.2
!pip install sentiment-analysis-spanish==0.0.25
!pip install gensim==4.3.0
!pip install pyLDAvis==3.4.1

In [ ]:
from sentiment_analysis_spanish import sentiment_analysis

In [ ]:
# Ejemplo 5.4.1
import gensim
from gensim.parsing.preprocessing import remove_stopwords, strip_punctuation, preprocess_string, strip_short, stem_text
from gensim import corpora

from gensim.models import LsiModel
from gensim.models.coherencemodel import CoherenceModel
from pprint import pprint

import matplotlib.pyplot as plt

import pyLDAvis
import pyLDAvis.gensim

In [ ]:
!python -m spacy download es_core_news_lg

In [ ]:
# Importación para semejanza semántica. Empezado a usar desde ejemplo 5.3.1
import spacy
nlp = spacy.load("es_core_news_lg")

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

<a name="intro"></a>
### Introducción

In [ ]:
"""
Lista de mensajes para pruebas. En general, estos mensajes se cargarán desde un 
fichero de texto y posiblemente estén almacenados con otros formatos, como JSON.
Ejemplo 5.1.1
"""
msjs_tor = ["Me encanta la tortilla de patata con cebolla",
            "No puedo creer que haya gente que tome la tortilla con cebolla",
            "sincebollista hasta la muerte, abajo la cebolla",
            "La cebolla no se toca",
            "Ojalá desaparezcan del mundo todos los cebollistas"]
msjs_vacas = ["Todos de vacaciones y yo 2 horas de atasco para ir a trabajar rodeado de vagos",
              "Qué bien se está en Madrid mientras todos están en la playa, todo para mí",
              "Mucho calor y la piscina petada y llena de gente gritando, que vuelva el invierno",
              "¡A disfrutar de playa y chiringuitos los próximos 15 días!",
              "¿Playa o montaña? ¡De las dos!"]
msjs = msjs_tor + msjs_vacas

<a name="sentimiento"></a>
### Análisis de sentimiento

<a name="sent_fun"></a>
#### Definición de funciones

In [ ]:
# sentiment_analysis_spanish
# Ejemplo 5.2.1
analizador_sap = sentiment_analysis.SentimentAnalysisSpanish()

In [ ]:
# pysentimient
# Ejemplo 5.2.7
from pysentimiento import create_analyzer

analizador_py_sent = create_analyzer(task="sentiment", lang="es")
analizador_py_odio = create_analyzer(task="hate_speech", lang="es")
analizador_py_emo = create_analyzer(task="emotion", lang="es")

In [ ]:
"""
Función que, dada una lista de frases, le asocia a cada una el sentimiento
calculado con la biblioteca sentiment_analysis_spanish.
"""
# Ejemplo 5.2.3
def asocia_sap(frases):
  lista_resultado = [(frase, analizador_sap.sentiment(frase)) for frase in frases]
  return lista_resultado

In [ ]:
"""
Función que, dada una lista de pares de la forma (frase, puntuación), calcula 
el sentimiento medio usando la biblioteca sentiment_analysis_spanish.
"""
# Ejemplo 5.2.5
def media_sap(pares):
  acumulador = 0
  for par in pares:
    medida_actual = par[1]
    acumulador = acumulador + medida_actual
  num_frases = len(pares)
  media = acumulador / num_frases
  return media

In [ ]:
"""
Función que, dada una lista de frases, calcula su sentimiento, su discurso de
odio y su emoción, y los integra en una tupla.
"""
# Ejemplo 5.2.13
def asocia_pysent(frases):
  lista_resultado = [(frase, analizador_py_sent.predict(frase).output, 
                             analizador_py_odio.predict(frase).output, 
                             analizador_py_emo.predict(frase).output) for frase in frases]
  return lista_resultado

<a name="sent_ejec"></a>
#### Ejecución

In [ ]:
# Ejemplo 5.2.2
analizador_sap.sentiment(msjs_tor[0])

In [ ]:
analizador_sap.sentiment(msjs_vacas[0])

In [ ]:
# Ejemplo 5.2.4
lista_pares_sap = asocia_sap(msjs_tor)
print(lista_pares_sap)

In [ ]:
# Ejemplo 5.2.6
media_sap(lista_pares_sap)

In [ ]:
# Ejemplo 5.2.8
analisis_sent = analizador_py_sent.predict(msjs_tor[0])
print(analisis_sent)

In [ ]:
analisis_sent.output

In [ ]:
analisis_sent.probas

In [ ]:
# Ejemplo 5.2.9
analisis_emo = analizador_py_emo.predict(msjs_tor[0])
print(analisis_emo)

In [ ]:
# Ejemplo 5.2.10
analisis_odio = analizador_py_odio.predict(msjs_tor[0])
print(analisis_odio)

In [ ]:
print(analizador_py_sent.predict(msjs_tor[0]))
print(analizador_py_odio.predict(msjs_tor[0]))
print(analizador_py_emo.predict(msjs_tor[0]))

In [ ]:
# Ejemplo 5.2.11
print(analizador_py_odio.predict("Te odio"))

In [ ]:
# Ejemplo 5.2.12
print(analizador_py_odio.predict("Ojalá desaparezcan todos los inmigrantes"))
print(analizador_py_odio.predict("Ojalá desaparezcan todas las mujeres"))
print(analizador_py_odio.predict("Ojalá desaparezcan todos los informáticos"))

In [ ]:
print(analizador_py_odio.predict("Ojalá desaparezcan del mundo todos los cebollistas"))

In [ ]:
asocia_pysent(msjs_tor)

In [ ]:
asocia_pysent(msjs_vacas)

<a name="semejanza"></a>
### Semejanza semántica

<a name="semejanza_fun"></a>
#### Definición de funciones

In [ ]:
"""
Dada una palabra, busca la palabra más parecida en una frase
"""
# Ejemplo 5.3.2
def palabra_parecida_en_frase(pal, frase):
  doc_pal = nlp(pal)
  doc_frase = nlp(frase)
  (max_token, max_simil) = (None, 0)
  for token in doc_frase:
    simil = doc_pal.similarity(token)
    if simil > max_simil:
      (max_token, max_simil) = (token, simil)
  return (max_token, max_simil)

In [ ]:
"""
Dada una palabra, busca la palabra más parecida en una lista de frases
"""
# Ejemplo 5.3.4
def palabra_parecida_en_frases(pal, frases):
  (max_frase, max_token, max_simil) = (None, None, 0)
  for frase in frases:
    (token, simil) = palabra_parecida_en_frase(pal, frase)
    if simil > max_simil:
      (max_frase, max_token, max_simil) = (frase, token, simil)
  return (max_frase, max_token, max_simil)

In [ ]:
"""
Dada una lista de palabras, busca la palabra más parecida a cada una de las palabras
en cada una de las frases
"""
# Ejemplo 5.3.6
def palabras_parecidas_en_frases(pals, frases):
  resultado = []
  for pal in pals:
    (frase, token, simil) = palabra_parecida_en_frases(pal, frases)
    res_pal = (pal, frase, token, simil)
    resultado.append(res_pal)
  return resultado

In [ ]:
# Ejemplo 5.3.9
def limpiar(frase):
  frase = frase.lower()
  doc = nlp(frase)
  lista_limpia = [token.text for token in doc if not token.is_space and not token.is_punct and not token.is_stop]
  return lista_limpia

In [ ]:
# Ejemplo 5.3.11
def doc_de_lista(lista_palabras):
  frase = " ".join(lista_palabras)
  doc = nlp(frase)
  return doc

In [ ]:
"""
Función que calcula la frase más parecida a una dada dentro de la lista. Todas las
frases pasan previamente por una fase de limpiado.
"""
# Ejemplo 5.3.13
def frases_parecidas(frase, frases):
  frase_limpia = limpiar(frase)
  doc_limpio = doc_de_lista(frase_limpia)
  (max_frase, max_simil) = (None, 0)
  for frase_actual in frases:
    actual_limpia = limpiar(frase_actual)
    actual_doc = doc_de_lista(actual_limpia)
    simil = doc_limpio.similarity(actual_doc)
    if simil > max_simil:
      (max_frase, max_simil) = (frase_actual, simil)
  return (max_frase, max_simil)

<a name="semejanza_ejec"></a>
#### Ejecución

In [ ]:
# Ejemplo 5.3.1
doc1 = nlp("sombrero")
doc2 = nlp("jabalí")
doc3 = nlp("zapato")
print(doc1.similarity(doc2))
print(doc1.similarity(doc3))
print(doc2.similarity(doc3))

In [ ]:
doc1 = nlp("alegre")
doc2 = nlp("triste")
print(doc1.similarity(doc2))

In [ ]:
doc1 = nlp("médico")
doc2 = nlp("diversión")
print(doc1.similarity(doc2))

In [ ]:
doc1 = nlp("teléfono")
doc2 = nlp("oxímoron")
print(doc1.similarity(doc2))

In [ ]:
# Ejemplo 5.3.3
print(palabra_parecida_en_frase("tubérculo", msjs_tor[0]))
print(palabra_parecida_en_frase("tubérculo", msjs_vacas[0]))

In [ ]:
# Ejemplo 5.3.5
palabra_parecida_en_frases("tubérculo", msjs)

In [ ]:
# Ejemplo 5.3.7
palabras_parecidas_en_frases(["tubérculo", "sombrero", "ornitorrinco"], msjs)

In [ ]:
# Ejemplo 5.3.8
doc1 = nlp(msjs_tor[0])
doc2 = nlp(msjs_vacas[0])
doc1.similarity(doc2)

In [ ]:
# Ejemplo 5.3.10
limpiar(msjs_tor[0])

In [ ]:
# Ejemplo 5.3.12
lista_limpia1 = limpiar(msjs_tor[0])
limpia1 = doc_de_lista(lista_limpia1)
lista_limpia2 = limpiar(msjs_vacas[0])
limpia2 = doc_de_lista(lista_limpia2)
limpia1.similarity(limpia2)

In [ ]:
# Ejemplo 5.3.14
frases_parecidas("Compre queso", msjs)

<a name="model"></a>
### Modelado de temas

<a name="model_fun"></a>
#### Definición de funciones

In [ ]:
# Ejemplo 5.4.2
def executeLSA(msjs, min_topics, max_topics):
  msjs_preparados = [limpiar(msj) for msj in msjs]
  dic = corpora.Dictionary(msjs_preparados)
  bow = [dic.doc2bow(text) for text in msjs_preparados]
  models = []
  coherences = []
  for num_topics in range(min_topics, max_topics + 1):
    lsa = LsiModel(bow, num_topics=num_topics, id2word=dic)
    coherence_model_lsa = CoherenceModel(model=lsa, texts=msjs_preparados, dictionary=dic, coherence='c_v')
    coherence_lsa = coherence_model_lsa.get_coherence()

    models.append(lsa)
    coherences.append(coherence_lsa)

  return (dic, coherences, models)

In [ ]:
# Ejemplo 5.4.4
def plot_graph(min_topics, max_topics, coherences, path):
  x = range(min_topics, max_topics + 1)
  plt.plot(x, coherences)
  plt.xlabel("Número de tópicos")
  plt.ylabel("Coherencia")
  plt.legend(("Valores de coherencia"), loc='best')
  plt.savefig(path)
  plt.show()

In [ ]:
# Ejemplo 5.4.7
def executeLDA(msjs, min_topics, max_topics, pasadas):
  msjs_preparados = [limpiar(msj) for msj in msjs]
  dic = corpora.Dictionary(msjs_preparados)
  corpus = [dic.doc2bow(text) for text in msjs_preparados]
  models = []
  coherences = []
  for num_topics in range(min_topics, max_topics + 1):
    lda = gensim.models.LdaMulticore(corpus=corpus, id2word=dic, num_topics=num_topics, passes=pasadas)
    coherence_model_lda = CoherenceModel(model=lda, texts=msjs_preparados, dictionary=dic, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()

    models.append(lda)
    coherences.append(coherence_lda)

  return (dic, corpus, coherences, models)

In [ ]:
# Ejemplo 5.4.14
def elegir_tema(lista_pesos):
  (topic, peso) = (-1, 0)
  for (curr_topic, curr_peso) in lista_pesos:
    if abs(curr_peso) > peso:
      (topic, peso) = (curr_topic, abs(curr_peso))
  return (topic, peso)

<a name="model_ejec"></a>
#### Ejecución

In [ ]:
# Ejemplo 5.4.3
(dic_lsa, coherencias_lsa, modelos_lsa) = executeLSA(msjs,2,10)

In [ ]:
# Ejemplo 5.4.5
plot_graph(2, 10, coherencias_lsa, "grafica_lsa.png")

In [ ]:
# Ejemplo 5.4.6
mi_modelo_lsa = modelos_lsa[1]
pprint(mi_modelo_lsa.print_topics(num_words=5))

In [ ]:
# Ejemplo 5.4.8
(dic_lda, corpus_lda, coherencias_lda, modelos_lda) = executeLDA(msjs,2,10,100)

In [ ]:
# Ejemplo 5.4.9
plot_graph(2, 10, coherencias_lda, "grafica_lda.png")

In [ ]:
# Ejemplo 5.4.10
mi_modelo_lda = modelos_lda[3]
pprint(mi_modelo_lda.print_topics(num_words=5))

In [ ]:
# Ejemplo 5.4.11
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(mi_modelo_lda, corpus_lda, dic_lda)

In [ ]:
# Ejemplo 5.4.12
nueva_frase = 'Quiero comer una buena tortilla en la playa.'
nueva_frase_limpia = limpiar(nueva_frase)

In [ ]:
# Ejemplo 5.4.13
bow_lsa_nuevo = dic_lsa.doc2bow(nueva_frase_limpia)
pesos_topics_lsa = mi_modelo_lsa[bow_lsa_nuevo]  
print(pesos_topics_lsa)

In [ ]:
# Ejemplo 5.4.15
(topic_lsa, peso_lsa) = elegir_tema(pesos_topics_lsa)
print(topic_lsa)

In [ ]:
# Ejemplo 5.4.16
bow_lda_nuevo = dic_lda.doc2bow(nueva_frase_limpia)
pesos_topics_lda = mi_modelo_lda[bow_lda_nuevo]  
print(pesos_topics_lda)

In [ ]:
# Ejemplo 5.4.17
(topic_lda, peso_lda) = elegir_tema(pesos_topics_lda)
print(topic_lda)

In [ ]:
# Pequeños ejemplos guiados para entender mejor el proceso
msjs_preparados = [limpiar(msj) for msj in msjs]

In [ ]:
msjs_preparados

In [ ]:
id2word = corpora.Dictionary(msjs_preparados)

In [ ]:
id2word[0]

In [ ]:
id2word[1]

In [ ]:
id2word[2]

In [ ]:
id2word[3]